In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Lab. Natural Lenguage Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [28]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [29]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("kg_train.csv",encoding='latin-1')

# Reduce the training set to speead up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [30]:
# Your code
# Let's divide the training and test set into two partitions
from sklearn.model_selection import train_test_split

X = data['text']
y = data['label']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

data_train = pd.DataFrame({'text': X_train, 'label': y_train})
data_val = pd.DataFrame({'text': X_val, 'label': y_val})

## Data Preprocessing

In [31]:
# Print some examples of punctuation and stop words
print(string.punctuation)
print(stopwords.words("english")[100:110])

# Create a SnowballStemmer object
snowball = SnowballStemmer('english')

# Remove inline JavaScript/CSS
data_train['preprocessed_text'] = data_train['text'].str.replace(r'<script.*?>.*?</script.*?>', '', flags=re.DOTALL, regex=True)
data_train['preprocessed_text'] = data_train['preprocessed_text'].str.replace(r'<style.*?>.*?</style.*?>', '', flags=re.DOTALL, regex=True)

# Remove html comments
data_train['preprocessed_text'] = data_train['preprocessed_text'].str.replace(r'<!--.*?-->', '', flags=re.DOTALL, regex=True)

# Remove remaining tags
data_train['preprocessed_text'] = data_train['preprocessed_text'].str.replace(r'<.*?>', '', flags=re.DOTALL, regex=True)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [33]:
# Remove all the special characters
data_train['preprocessed_text'] = data_train['preprocessed_text'].str.replace(r'[^a-zA-Z\s]', '', regex=True)

# Remove numbers
data_train['preprocessed_text'] = data_train['preprocessed_text'].str.replace(r'\d+', '', regex=True)

# Remove all single characters
data_train['preprocessed_text'] = data_train['preprocessed_text'].str.replace(r'\s+[a-zA-Z]\s+', ' ', regex=True)

# Remove single characters from the start
data_train['preprocessed_text'] = data_train['preprocessed_text'].str.replace(r'\^[a-zA-Z]\s+', ' ', regex=True)

# Substituting multiple spaces with single space
data_train['preprocessed_text'] = data_train['preprocessed_text'].str.replace(r'\s+', ' ', regex=True)

# Remove prefixed 'b'
data_train['preprocessed_text'] = data_train['preprocessed_text'].str.replace(r'^b\s+', '', regex=True)

# Convert to Lowercase
data_train['preprocessed_text'] = data_train['preprocessed_text'].str.lower()

## Now let's work on removing stopwords
Remove the stopwords.

In [34]:
# Remove the stopwords
stop_words = set(stopwords.words('english'))
data_train['preprocessed_text'] = data_train['preprocessed_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [35]:
# Tame Your Text with Lemmatization
lemmatizer = WordNetLemmatizer()
data_train['preprocessed_text'] = data_train['preprocessed_text'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))

## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [36]:
# Bag Of Words
from sklearn.feature_extraction.text import CountVectorizer

# Create a CountVectorizer object
vectorizer = CountVectorizer()

# Fit and transform the data
X_train = vectorizer.fit_transform(data_train['preprocessed_text'])

# Get the feature names
feature_names = vectorizer.get_feature_names_out()

# Get the top 10 words in ham and spam messages
ham_counts = X_train[data_train['label'] == 'ham'].sum(axis=0).A1
spam_counts = X_train[data_train['label'] == 'spam'].sum(axis=0).A1

ham_top_words = sorted([(feature_names[i], ham_counts[i]) for i in ham_counts.nonzero()[0]], key=lambda x: x[1], reverse=True)[:10]
spam_top_words = sorted([(feature_names[i], spam_counts[i]) for i in spam_counts.nonzero()[0]], key=lambda x: x[1], reverse=True)[:10]

print("Top 10 words in ham messages:")
for word, count in ham_top_words:
    print(f"{word}: {count}")

print("\nTop 10 words in spam messages:")
for word, count in spam_top_words:
    print(f"{word}: {count}")

Top 10 words in ham messages:

Top 10 words in spam messages:


In [39]:
# Remove inline JavaScript/CSS
data_val['preprocessed_text'] = data_val['text'].str.replace(r'<script.*?>.*?</script.*?>', '', flags=re.DOTALL, regex=True)
data_val['preprocessed_text'] = data_val['preprocessed_text'].str.replace(r'<style.*?>.*?</style.*?>', '', flags=re.DOTALL, regex=True)

# Remove html comments
data_val['preprocessed_text'] = data_val['preprocessed_text'].str.replace(r'<!--.*?-->', '', flags=re.DOTALL, regex=True)

# Remove remaining tags
data_val['preprocessed_text'] = data_val['preprocessed_text'].str.replace(r'<.*?>', '', flags=re.DOTALL, regex=True)

# Remove all the special characters
data_val['preprocessed_text'] = data_val['preprocessed_text'].str.replace(r'[^a-zA-Z\s]', '', regex=True)

# Remove numbers
data_val['preprocessed_text'] = data_val['preprocessed_text'].str.replace(r'\d+', '', regex=True)

# Remove all single characters
data_val['preprocessed_text'] = data_val['preprocessed_text'].str.replace(r'\s+[a-zA-Z]\s+', ' ', regex=True)

# Remove single characters from the start
data_val['preprocessed_text'] = data_val['preprocessed_text'].str.replace(r'\^[a-zA-Z]\s+', ' ', regex=True)

# Substituting multiple spaces with single space
data_val['preprocessed_text'] = data_val['preprocessed_text'].str.replace(r'\s+', ' ', regex=True)

# Remove prefixed 'b'
data_val['preprocessed_text'] = data_val['preprocessed_text'].str.replace(r'^b\s+', '', regex=True)

# Convert to Lowercase
data_val['preprocessed_text'] = data_val['preprocessed_text'].str.lower()

# Remove the stopwords
stop_words = set(stopwords.words('english'))
data_val['preprocessed_text'] = data_val['preprocessed_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

# Tame Your Text with Lemmatization
lemmatizer = WordNetLemmatizer()
data_val['preprocessed_text'] = data_val['preprocessed_text'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))

## Extra features

In [41]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words)
money_simbol_list = "|".join(["euro", "dollar", "pound", "€", "$"])
suspicious_words = "|".join(["free", "cheap", "sex", "money", "account", "bank", "fund", "transfer", "transaction", "win", "deposit", "password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list, regex=True, na=False) * 1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words, regex=True, na=False) * 1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x))

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list, regex=True, na=False) * 1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words, regex=True, na=False) * 1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x))

data_train.head()

,text,label,preprocessed_text,money_mark,suspicious_words,text_len
29,"----------- REGARDS, MR NELSON SMITH.KINDLY RE...",1,regard mr nelson smithkindly reply private ema...,1,0,75
535,I have not been able to reach oscar this am. W...,0,able reach oscar supposed send pdb receive,1,0,42
695,; Huma Abedin B6I'm checking with Pat on the 5...,0,huma abedin bim checking pat work jack jake re...,1,0,79
557,I can have it announced here on Monday - can't...,0,announced monday cant today,1,0,27
836,BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 S...,1,bank africaagence san pedro bp san pedro cote ...,1,1,1067


## How would work the Bag of Words with Count Vectorizer concept?

In [42]:
# Bag of Words with Count Vectorizer
from sklearn.feature_extraction.text import CountVectorizer

# Create a CountVectorizer object
vectorizer = CountVectorizer()

# Fit and transform the data
X_train = vectorizer.fit_transform(data_train['preprocessed_text'])
X_val = vectorizer.transform(data_val['preprocessed_text'])

# Get the feature names
feature_names = vectorizer.get_feature_names_out()

print(f"Shape of the vectorized training data: {X_train.shape}")
print(f"Shape of the vectorized validation data: {X_val.shape}")

Shape of the vectorized training data: (800, 16927)
Shape of the vectorized validation data: (200, 16927)


## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [43]:
# Your code
# TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

# Load the vectorizer
vectorizer = TfidfVectorizer()

# Vectorize all dataset
X_train = vectorizer.fit_transform(data_train['preprocessed_text'])
X_val = vectorizer.transform(data_val['preprocessed_text'])

# Print the shape of the vectorized dataset
print(f"Shape of the vectorized training data: {X_train.shape}")
print(f"Shape of the vectorized validation data: {X_val.shape}")

Shape of the vectorized training data: (800, 16927)
Shape of the vectorized validation data: (200, 16927)


## And the Train a Classifier?

In [46]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Create a Multinomial Naive Bayes classifier
clf = MultinomialNB()

# Train the classifier
clf.fit(X_train, data_train['label'])

# Make predictions on the validation set
y_pred = clf.predict(X_val)

# Calculate accuracy and print classification report
accuracy = accuracy_score(data_val['label'], y_pred)
print(f"Accuracy: {accuracy:.4f}")
print(classification_report(data_val['label'], y_pred))

Accuracy: 0.9300
              precision    recall  f1-score   support

           0       0.99      0.90      0.94       125
           1       0.85      0.99      0.91        75

    accuracy                           0.93       200
   macro avg       0.92      0.94      0.93       200
weighted avg       0.94      0.93      0.93       200



### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [47]:
# Your code
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

# Create a pipeline with TfidfVectorizer and MultinomialNB
pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('classifier', MultinomialNB())
])

# Train the pipeline
pipeline.fit(data_train['preprocessed_text'], data_train['label'])

# Make predictions on the validation set
y_pred = pipeline.predict(data_val['preprocessed_text'])

# Calculate accuracy and print classification report
accuracy = accuracy_score(data_val['label'], y_pred)
print(f"Accuracy: {accuracy:.4f}")
print(classification_report(data_val['label'], y_pred))

Accuracy: 0.9300
              precision    recall  f1-score   support

           0       0.99      0.90      0.94       125
           1       0.85      0.99      0.91        75

    accuracy                           0.93       200
   macro avg       0.92      0.94      0.93       200
weighted avg       0.94      0.93      0.93       200



In [ ]:
#In this extra task, we're implementing a SPAM/HAM classifier using the Multinomial Naive Bayes classifier with default parameters. We're not allowed to change the classifier,
but we can work on finding the best feature representation.
#We're creating a pipeline that combines the TfidfVectorizer and the MultinomialNB classifier. The pipeline allows us to chain multiple estimators together and perform the
necessary data transformations and model fitting in a single step.
#We're fitting the pipeline on the training data using fit, which learns the vocabulary and the parameters of the Naive Bayes model.
#Next, we're making predictions on the validation data using predict, which returns the predicted labels (spam or ham) for each text entry.
#Finally, we're calculating the accuracy score using accuracy_score from scikit-learn and printing the classification report, which shows the precision, recall, and F1-score 
for each class.
#You can experiment with different feature representations, such as adding or removing the extra features we created earlier (money_mark, suspicious_words, text_len),
or trying different preprocessing techniques to improve the performance of the classifier.
